In [1]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [2]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    node=CustomNode('<eps>',state_number,'-',False,root_node)
    state_number+=1
    node=CustomNode('^',state_number,'<eps>',False,node)
    for ch in suffixes[0][1:]:
        state_number+=1
        node=CustomNode(ch,state_number,ch,False,node)
    state_number+=1
    CustomNode('<eps>',state_number,suffixes[1],True,node)

    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode("^s",state_number,'+PL',True,node)
        state_number+=1
        CustomNode("'s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+3D',True,node)
        state_number+=1
        CustomNode("^ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_noun(stem,state_number,root):
    top=root
    verbs=stems=stem.split('/')
    for i in range(len(verbs[0])):
        state_number+=1
        root=CustomNode(verbs[0][i],state_number,verbs[0][i],False,root)
    end_ptr=root
   
    
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+N',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+SG',True,end_ptr)
   
    end_ptr=top
    for i in range(len(verbs[0])):
        state_number+=1
        end_ptr=CustomNode('<eps>',state_number,verbs[0][i],False,end_ptr)
   
    
    for i in range(len(verbs[1])):
        state_number+=1
        end_ptr=CustomNode(verbs[1][i],state_number,'<eps>',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+N',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+PL',True,end_ptr)
            
    return (root,state_number)

In [3]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Adjectives.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        print(line)
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_noun(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            
            state_number+=1
            end_point=CustomNode("<eps>",state_number,'+N',False,root)
            state_number+=1
            CustomNode("<eps>",state_number,'+SG',True,end_point)
            state_number+=1
            CustomNode("^s",state_number,'+PL',True,end_point)
        
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]


awkward (-ly:+ADV|-ness:+N)
bold (-ly:+ADV|-ness:+N|-er:+CP|-st:+SP)
brave (-ly:+ADV|-ry:+N|-ness:+N|-er:+CP|-st:+SP)
bright (-en:+V|-ish:+ADJ|-ness:+N|-ly:+ADV|-er:+CP|-st:+SP)
defiant (-ce:+N|-ly:+ADV)
deft (-ly:+ADV| -ness:N)
deliberate (-ly:+ADV|-ness:+N)
eager (-ly:+ADV|-ness:+N)
elegant (-ce:+N|-ly:+ADV)
exact
final
frequent (-cy:+ADJ|-ation:+N|-ative:+ADJ|-er:+N|-ly:+ADV)
fresh (-en:+V(-er:+N)|-ly:+N)
happy (-ly:+ADV|-ness:+N|-er:+CP|-st:+SP)
honest (-y:+N|-ly:+ADV|-ness:+N)
hungry (-ly:+ADV|-ness:+N)
innocent
just (-ify:+V|-ly:+ADV|-ice:+N)
kind(-ly:+ADV)
lazy (-ly:+ADV|-ness:+N|-er:+CP|-st:+SP)
mad (-ly:+ADV|-ness:+N)
merry (-ly:+ADV|-ness:+N)
mortal
nerve
obnoxious (-ly:+ADV|-ness:+N)
polite (-ly:+ADV|-ness:+N)
poor (-ly:+ADV(-ness:+N))
rapid (-ly:+ADV|-ness:+N)
rare (-ly:+ADV|-ness:+ADJ)
rude (-ly:+ADV|-ness:N)
safe (-ly:+ADV|-ness:+N|-er:+CP|-st:+SP)
serious (-ly:+ADV|-ness:+N)
sharp (-ly:+ADV|-en:+V(-er:+N)|-er:+CP|-st:+SP)
slow (-er:+CP|-st:+SP)
solemn (-ly:+ADV|-ity:+N|-

In [4]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

a 1parent: root
w 2parent: a
k 3parent: w
w 4parent: k
a 5parent: w
r 6parent: a
d 7parent: r
<eps> 8parent: d
<eps> 9parent: <eps>
^s 10parent: <eps>
<eps> 11parent: d
^ 12parent: <eps>
l 13parent: ^
y 14parent: l
<eps> 15parent: y
<eps> 16parent: d
^ 17parent: <eps>
n 18parent: ^
e 19parent: n
s 20parent: e
s 21parent: s
<eps> 22parent: s
^s 23parent: s
' 25parent: ^s
's 24parent: s
b 26parent: root
o 27parent: b
l 28parent: o
d 29parent: l
<eps> 30parent: d
<eps> 31parent: <eps>
^s 32parent: <eps>
<eps> 33parent: d
^ 34parent: <eps>
l 35parent: ^
y 36parent: l
<eps> 37parent: y
<eps> 38parent: d
^ 39parent: <eps>
n 40parent: ^
e 41parent: n
s 42parent: e
s 43parent: s
<eps> 44parent: s
^s 45parent: s
' 47parent: ^s
's 46parent: s
<eps> 48parent: d
^ 49parent: <eps>
e 50parent: ^
r 51parent: e
<eps> 52parent: r
<eps> 53parent: d
^ 54parent: <eps>
s 55parent: ^
t 56parent: s
<eps> 57parent: t
r 58parent: b
a 59parent: r
v 60parent: a
e 61parent: v
<eps> 62parent: e
<eps> 63parent: <ep

In [5]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	a	a
4	5	w	w
5	6	k	k
6	7	w	w
7	8	a	a
8	9	r	r
9	10	d	d
10	11	+N	<eps>
11	12	+SG	<eps>
12	0	<eps>	#
11	13	+PL	^s
13	0	<eps>	#
10	14	-	<eps>
14	15	<eps>	^
15	16	l	l
16	17	y	y
17	18	+ADV	<eps>
18	0	<eps>	#
10	19	-	<eps>
19	20	<eps>	^
20	21	n	n
21	22	e	e
22	23	s	s
23	24	s	s
24	25	+N	<eps>
25	0	<eps>	#
24	26	+PL	^s
26	0	<eps>	#
26	28	+PLPS	'
28	0	<eps>	#
24	27	+SGPS	's
27	0	<eps>	#
0	29	b	b
29	30	o	o
30	31	l	l
31	32	d	d
32	33	+N	<eps>
33	34	+SG	<eps>
34	0	<eps>	#
33	35	+PL	^s
35	0	<eps>	#
32	36	-	<eps>
36	37	<eps>	^
37	38	l	l
38	39	y	y
39	40	+ADV	<eps>
40	0	<eps>	#
32	41	-	<eps>
41	42	<eps>	^
42	43	n	n
43	44	e	e
44	45	s	s
45	46	s	s
46	47	+N	<eps>
47	0	<eps>	#
46	48	+PL	^s
48	0	<eps>	#
48	50	+PLPS	'
50	0	<eps>	#
46	49	+SGPS	's
49	0	<eps>	#
32	51	-	<eps>
51	52	<eps>	^
52	53	e	e
53	54	r	r
54	55	+CP	<eps>
55	0	<eps>	#
32	56	-	<eps>
56	57	<eps>	^
57	58	s	s
58	59	t	t
59	60	+SP	<eps>
60	0	<eps>	#
29	61	r	r


In [33]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)